In [1]:
import cvxpy as cvx
import numpy as np

In [3]:

def getCycleTime(t,l,L,W):
    # Get cycle time
    # t: list of resolvent times for each agent
    # l: n x n array of communication times
    # L: n x n array of resolvent multipliers
    # W: n x n array of consensus multipliers
    # Uses critical path method
    n = len(t)
    # X[i,j] is 1 if L[i,j] is nonzero, i>j
    # X[j,i] is 1 if W[i,j] is nonzero, i<j
    X = np.zeros((n,n))
    for i in range(n):
        for j in range(i):
            if L[i,j] != 0:
                X[i,j] = 1
            if W[i,j] != 0:
                X[j,i] = 1
    # Declare variables
    itrs = 3*n-1
    s = cvx.Variable((itrs,n), nonneg=True) # completion time for each agent in each iteration

    M = itrs*(max(t) + np.max(l))*n
    # Build constraints
    cons = [ s[i, k] - s[i, j] >= (t[j] + l[j,k] + M)*X[k,j] - M for i in range(itrs) for j in range(n-1) for k in range(j+1, n)]
    cons += [s[i+1, k] - s[i, j] >= (t[j] + l[j,k] + M)*X[j,k] - M for i in range(itrs-1) for k in range(1,n) for j in range(k)]
    cons += [s[i+1, j] - s[i, k] >= (t[k] + l[k,j] + M)*X[j,k] - M for i in range(itrs-1) for k in range(1,n) for j in range(k)]

    # Build objective
    obj = cvx.Minimize(cvx.max(s[itrs-1,:]))

    # Solve problem
    prob = cvx.Problem(obj, cons)
    prob.solve()
    return prob.value/itrs

# W = np.array([[1, -1, 0], [-1, 2, -1], [0, -1, 1]])
# L = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0]])
# t = [1, 1, 1]
# l = np.ones((3,3))/10
# print(getCycleTime(t,l,L,W))

def getRyu(n):
    '''Get Ryu values for W and L
    n: number of agents'''
    W = np.eye(n)
    # Set bottom row to -np.ones
    W[n-1,:] = -np.ones(n)
    W[:,n-1] = -np.ones(n)
    W[n-1,n-1] = n-1
    W = 2/(n-1)*W
    L = np.zeros((n,n))
    # Set lower triangle of L to 1
    for i in range(n):
        for j in range(i):
            L[i,j] = 1
    L = 2/(n-1)*L
    return W, L

def getMT(n):
    '''Get Malitsky-Tam values for W and L
    n: number of agents'''
    W = np.zeros((n,n))
    W[0,0] = 1
    W[0,1] = -1
    for r in range(1,n-1):
        W[r,r-1] = -1
        W[r,r] = 2
        W[r,r+1] = -1
    W[n-1,n-1] = 1
    W[n-1,n-2] = -1

    L = np.zeros((n,n))
    # Add ones just below the diagonal
    for i in range(n-1):
        L[i+1,i] = 1
    L[n-1,0] = 1
    return W, L



In [4]:
L_mt, W_mt = getMT(4)
print(L_mt, W_mt)

[[ 1. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  1.]] [[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]


In [5]:
t = [3,1,1,1]
l = np.ones((4,4))/10

print(getCycleTime(t,l,L_mt,W_mt))

69.29999999999976


In [6]:
L_b = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [1.5, 0.5, 0, 0], [0.5,1.5,0,0]])
W_b = np.array([[1, 0, -1, 0], [0, 2, -0.5, -1.5], [-1, -0.5, 1.67218, -0.17218], [0,-1.5,-0.17218,1.67218]])

In [7]:
print(getCycleTime(t,l,L_b,W_b))

45.09999999999988
